# Object Classifier

## Process images

### Extract data from .7z files

**Instructions**

1. Download the dataset from the following link: https://www.kaggle.com/c/cifar-10
2. Extract the dataset into a directory named `data`.
3. Run this notebook.


In [ ]:
from zipfile import ZipFile
dataset = ZipFile('data/cifar-10.zip')
dataset.extractall(path='data')
dataset.close()
print("Dataset extracted successfully")

Dataset extracted successfully


In [ ]:
import py7zr

archive = py7zr.SevenZipFile('data/train.7z', mode='r')
archive.extractall(path='data')
archive.close()
train_file_location='data/train'

archive = py7zr.SevenZipFile('data/test.7z', mode='r')
archive.extractall(path='data')
archive.close()
test_file_location='data/test'


In [3]:
#imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [4]:
filenames = os.listdir("data/train")
print(len(filenames))

50000


### Process Data

In [15]:
def get_image_as_array(path):
    try:
        image = Image.open(path)
        return np.array(image)
    except:
        return None

df = pd.read_csv('data/trainLabels.csv')
df['image'] = df['id'].apply(lambda x: get_image_as_array(f'data/train/{x}.png'))





In [40]:
df.head()

,id,label,image
0,1,0,"[[[0.23137254901960785, 0.24313725490196078, 0..."
1,2,1,"[[[0.6039215686274509, 0.6941176470588235, 0.7..."
2,3,1,"[[[1.0, 1.0, 1.0], [0.9921568627450981, 0.9921..."
3,4,2,"[[[0.10980392156862745, 0.09803921568627451, 0..."
4,5,3,"[[[0.6666666666666666, 0.7058823529411765, 0.7..."


In [17]:
df.dropna(inplace=True, subset=['image'])

In [ ]:
df.shape

(49999, 3)

In [19]:
object_classes = df['label'].unique()
print(object_classes)

['frog' 'truck' 'deer' 'automobile' 'bird' 'horse' 'ship' 'cat' 'dog'
 'airplane']


In [20]:


entries_per_class = df['label'].value_counts()
print(entries_per_class) # no need to standardize or scale

label
frog          5000
truck         5000
deer          5000
bird          5000
horse         5000
ship          5000
cat           5000
dog           5000
airplane      5000
automobile    4999
Name: count, dtype: int64


In [21]:
# labels_dictionary = dict(enumerate(object_classes))
# print(labels_dictionary)
labels_map = {}
for index, item in enumerate(object_classes):
    labels_map[item] = index

print(labels_map)

{'frog': 0, 'truck': 1, 'deer': 2, 'automobile': 3, 'bird': 4, 'horse': 5, 'ship': 6, 'cat': 7, 'dog': 8, 'airplane': 9}


In [22]:
df['label'] = df['label'].map(labels_map)
df['image'] = df['image']/255
df.head()
df.shape

(49999, 3)

In [23]:
num_rows_80_percent = int(len(df) * 0.8)

train = df.iloc[:num_rows_80_percent]
test = df.iloc[num_rows_80_percent:]

print(train.shape)
print(test.shape)


(39999, 3)
(10000, 3)


In [41]:
X_train = np.stack(X_train).astype("float32")   # shape: (N, 32, 32, 3)
X_test  = np.stack(X_test).astype("float32")

Y_train = np.array(Y_train, dtype="int32")      # shape: (N,)
Y_test  = np.array(Y_test, dtype="int32")

print("X_train.shape: ", X_train.shape)
print("Y_train.shape: ", Y_train.shape)

print("X_test.shape: ", X_test.shape)
print("Y_test.shape: ", Y_test.shape)





X_train.shape:  (39999, 32, 32, 3)
Y_train.shape:  (39999,)
X_test.shape:  (10000, 32, 32, 3)
Y_test.shape:  (10000,)


## Create Neural Net

We shall create the Neural Network

In [28]:
import tensorflow as tf
from tensorflow import keras



/Users/ansinha/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


#### Setting up layers

In [42]:
num_of_classes = len(object_classes)

#setting up layers

inputs = keras.Input(shape=(32, 32, 3))
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(64, activation='relu')(x)
outputs = keras.layers.Dense(num_of_classes, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [43]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', # The data is not one hot encoded
    metrics=['accuracy'])

In [45]:
history=model.fit(X_train,Y_train, validation_split=0.2, epochs=10)

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3447 - loss: 1.8058 - val_accuracy: 0.3451 - val_loss: 1.8013
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3502 - loss: 1.7913 - val_accuracy: 0.3415 - val_loss: 1.7957
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3505 - loss: 1.7855 - val_accuracy: 0.3479 - val_loss: 1.7936
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3579 - loss: 1.7685 - val_accuracy: 0.3426 - val_loss: 1.8075
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3579 - loss: 1.7605 - val_accuracy: 0.3359 - val_loss: 1.8207
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3612 - loss: 1.7613 - val_accuracy: 0.3492 - val_loss: 1.7888
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3542 - loss: 1.7618 - val_accuracy: 0.3481 - val_loss: 1.8070
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3649 - loss: 1.7533 - 

## Using Transfer Learning

In [ ]:
convolutional_base = ResNet50